## 导入必要库

In [16]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 加载并预处理 MNIST 数据集

MNIST 包含 60000 张训练图像和 10000 张测试图像，每张为 28×28 像素的手写数字（0-9）

代码通过datasets.MNIST自动下载数据，并通过transforms.ToTensor()将图像转为张量（维度为[1,28,28]）

用DataLoader按批次加载数据（批次大小batch_size=2048，训练集打乱数据顺序）

In [10]:
transform = transforms.Compose([transforms.ToTensor()])  # 转为张量

# 加载训练集和测试集
train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

## 定义 MLP 模型
模型为两层全连接神经网络，结构：输入层（784 维，28×28 展平）→ 隐藏层（128 维，ReLU 激活）→ 输出层（10 维，对应 10 个数字）。

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(784, 128)  # 第一层全连接：784→128
        self.l2 = nn.Linear(128, 10)   # 第二层全连接：128→10
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # 展平图像：[batch,1,28,28]→[batch,784]
        x = torch.relu(self.l1(x))  # 隐藏层+ReLU激活
        x = self.l2(x)              # 输出层（无激活，后续用交叉熵损失）
        return x

model = MLP()  # 初始化模型

## 设置训练参数
损失函数：交叉熵损失（nn.CrossEntropyLoss，适用于分类任务）。

优化器：随机梯度下降（SGD，学习率lr=0.1）。

训练轮次：epochs=10

In [12]:
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)  # SGD优化器
epochs = 10  # 训练轮次

## 训练模型 
每轮遍历训练集，通过前向传播计算预测值，反向传播更新参数：

In [14]:
for epoch in range(epochs):
    model.train()  # 训练模式
    train_loss = 0.0
    
    for images, labels in train_loader:
        optimizer.zero_grad()  # 清零梯度
        outputs = model(images)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播（计算梯度）
        optimizer.step()  # 更新参数
        train_loss += loss.item() * images.size(0)  # 累计损失
    
    # 计算平均训练损失
    train_loss_avg = train_loss / len(train_dataset)

## 测试模型
每轮训练后，在测试集上评估模型性能（损失和准确率）

In [15]:
model.eval()  # 评估模式（关闭 dropout 等）
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # 关闭梯度计算，节省资源
    for images, labels in test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        
        # 计算准确率（取输出最大值对应的类别为预测结果）
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss_avg = test_loss / len(test_dataset)
test_acc = 100 * correct / total

# 输出每轮结果
print(f"Epoch {epoch+1}/{epochs}")
print(f"Train Loss: {train_loss_avg:.4f} | Test Loss: {test_loss_avg:.4f} | Test Acc: {test_acc:.2f}%")

Epoch 10/10
Train Loss: 0.2981 | Test Loss: 0.2855 | Test Acc: 91.89%
